# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686517


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:41,  3.50s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:10,  2.50s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:46,  1.78s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:32,  1.29s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  5.78it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  6.04it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  6.58it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  6.43it/s]

ElisabethB                            0.000531
not-logged-in-20a275b53abec8db9f41    0.029572
not-logged-in-e385081e7d0d82fc95d4    0.000992
jammiedodgers                         0.001401
RockyGeo                              0.000470
CThomas                               0.000260
mqthompson                            0.000220
Dynamatt                              0.000131
kjkavene                              0.000083
kawaboy                               0.000507
not-logged-in-5f30b2bb9b7f9d49da41    0.028870
AdamMorse                             0.000144
DarioB                                0.000110
not-logged-in-214814e52004b42491a4    2.936603
pangeli5                              0.001800
not-logged-in-2baba15765cafb4ba245    0.000460
not-logged-in-04ed9ae880f18762cd9e    0.180805
Lavadude                              0.026260
jordanrushworth                       0.001692
not-logged-in-2027f1f6703c19b00bd8    0.000962
Milan05                               0.000215
Peyton.Harvey

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())